In [1]:
#instaluje wszystkie niezbedne biblioteki
from bs4 import BeautifulSoup
import requests
import json

In [2]:
url = 'https://www.carrefour.pl/'

headers = {
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
}


req = requests.get(url,headers)
src1 = req.text

In [3]:
with open("carrefour_page.html", "w") as file:
    file.write(src1)
    
with open("carrefour_page.html") as file:
    scr2 = file.read()

In [4]:
soup = BeautifulSoup(scr2,"lxml")

In [5]:
all_categories_dict = {}
all_categories_name = soup.find_all("a", class_ = "gtm-cat")
for item in all_categories_name:
    item_text = item.text
    item_hrefs = "https://www.carrefour.pl" + item.get("href")
    all_categories_dict[item_text] = item_hrefs

In [6]:
with open("categories_carrefour.json", 'w') as file:
    json.dump(all_categories_dict,file, ensure_ascii=False,indent=4)

with open("categories_carrefour.json") as file:
    all_categories_dict = json.load(file)

In [7]:
all_categories_dict

{'Artykuły spożywcze': 'https://www.carrefour.pl/artykuly-spozywcze',
 'Mleko, nabiał, jaja': 'https://www.carrefour.pl/mleko-nabial-jaja',
 'Napoje': 'https://www.carrefour.pl/napoje',
 'Drogeria, kosmetyki i zdrowie': 'https://www.carrefour.pl/drogeria-kosmetyki-i-zdrowie',
 'Zdrowa żywność': 'https://www.carrefour.pl/zdrowa-zywnosc',
 'Owoce, warzywa, zioła': 'https://www.carrefour.pl/owoce-warzywa-ziola',
 'Mięso': 'https://www.carrefour.pl/mieso',
 'Mrożonki': 'https://www.carrefour.pl/mrozonki',
 'Wędliny, kiełbasy': 'https://www.carrefour.pl/wedliny-kielbasy',
 'Piekarnia, ciastkarnia': 'https://www.carrefour.pl/piekarnia-ciastkarnia',
 'Ryby i owoce morza': 'https://www.carrefour.pl/ryby-i-owoce-morza',
 'Dziecko': 'https://www.carrefour.pl/dziecko',
 'Dla zwierząt': 'https://www.carrefour.pl/dla-zwierzat',
 'Dania gotowe i przystawki': 'https://www.carrefour.pl/dania-gotowe-i-przystawki',
 'Zakupy niecodzienne': 'https://www.carrefour.pl/zakupy-niecodzienne'}

In [103]:
count = 0
for caterogies_name, categories_href in all_categories_dict.items():
    if count < 1:
        req = requests.get(categories_href,headers = headers)
        scr_pages = req.text
        
        with open(f"data_carrefour/{count}_{caterogies_name}.html", "w") as file:
            file.write(scr_pages)
        
        with open(f"data_carrefour/{count}_{caterogies_name}.html") as file:
            scr_page = file.read()
        
        soup = BeautifulSoup(scr_page,'lxml')
        
        all_subcategories_dict = {}
        subcategories_names = soup.find_all("p", class_ = "MuiTypography-alignLeft")
        subcategories_hrefs = soup.find_all("a", class_ = "MuiButton-fullWidth")
        
        all_subcategories_names = []
        for subcategory_name in subcategories_names:
            subcategory_name = subcategory_name.text
            all_subcategories_names.append(subcategory_name)
        
        all_subcategories_hrefs = []
        for subcategory_href in subcategories_hrefs:
            subcategory_href = "https://www.carrefour.pl" + subcategory_href.get("href")
            all_subcategories_hrefs.append(subcategory_href)
            
        
        #usywam dwa ostatnie elementy "Lista kategorii', 'Ustawienia cookies" z listy
        all_subcategories_names = all_subcategories_names[:-2]
        
        #usywam pierwszy elementy z listy linkow
        all_subcategories_hrefs = all_subcategories_hrefs[1:]
        
        #tworze slownik w za pomoca dwoch list
        all_subcategories_dict = {all_subcategories_names[i]: all_subcategories_hrefs[i] \
                                  for i in range(len(all_subcategories_names))}
        
        
        with open(f"data_carrefour/json_categories/{count}_{caterogies_name}.json", 'w') as file:
            json.dump(all_subcategories_dict,file, ensure_ascii=False,indent=4)
        
        count_sub = 0
        for subcategories_names,subcategories_hrefs in all_subcategories_dict.items():
            if count_sub < 3:
                url = subcategories_hrefs
                req = requests.get(url,headers = headers)
                src_subcategories = req.text

                with open(f"data_carrefour/html sub/{count}_{caterogies_name}.html", "w") as file:
                    file.write(src_subcategories)

                with open(f"data_carrefour/html sub/{count}_{caterogies_name}.html") as file:
                    src_subcategories = file.read()

                soup_subcat = BeautifulSoup(src_subcategories,'lxml')

                product_name = soup_subcat.find_all("a", class_ = "jss322")
                # products_price_zl = soup_subcat.find_all("div",class_ = 'MuiTypography-h1')
                # products_price_grosz = soup_subcat.find_all("div", class_ = "MuiTypography-h3")
                
                # for products_zl in products_price_zl:
                #   product_price_zl = products_zl.text
                #   print(product_price)
                
                for product in product_name:
                    product_name = product.text
                    print(product_name)
                    
            count_sub += 1
    count += 1

EkoWital Ekologiczna passata pomidorowa 680 g
Mlekovita Twaróg bez laktozy półtłusty 230 g
Mlekovita Wydojone Mleko bez laktozy 3,2% 500 ml
Finuu Miks tłuszczowy do smarowania bez laktozy 200 g
Mlekovita Gouda Ser bez laktozy w plastrach 150 g
BioSaurus Ekologiczne pieczone chrupki kukurydziane o smaku ketchupowym 50 g
Mlekovita Sokół Ser bez laktozy w plastrach 150 g
Mlekovita Masło Polskie ekstra bez laktozy 82% 200 g
BioSaurus Ekologiczne pieczone chrupki kukurydziane o smaku soli morskiej 50 g
Mlekovita Serek homogenizowany Polski bez laktozy waniliowy 150 g
Masło ekstra z Łukowa bez laktozy 200 g
Mamut Biszkopty wrocławskie bez dodatku cukrów 100 g
Łaciaty Jogurt pitny jeżyna malina bez laktozy 250 ml
Robico Kefir bez laktozy 1,5% 400 g
Melvit Płatki owsiane górskie 500 g
Carrefour Bio Syrop z agawy 350 g
BioSaurus Ekologiczne pieczone chrupki kukurydziane o smaku serowym 50 g
Łaciaty Jogurt pitny ananas kokos bez laktozy 250 ml
Sante Kotlety sojowe a la schabowe 100 g
Sante Otręb